In [ ]:
import tarfile
import json
import bz2
import pandas as pd
import time
import os
import matplotlib.pyplot as plt
import re

Unpack all files and write them into a single dataframe (mother_df)

In [ ]:
file_location = './data_unpacked/'
dirListing = os.listdir(file_location)
mother_df = pd.DataFrame()


for subdir in dirListing:
    for file in os.listdir(file_location + subdir  + '/'):
            
        existing_file_path = os.path.join(file_location, subdir, file)

        new_file_path = os.path.join(file_location, subdir, file)

        l = []
        with open(new_file_path) as f:
            for line in f:
                l.append(json.loads(line))
        df = pd.DataFrame(l[0:])
        # Betfair time is in milliseconds, so this converts it to normal Unix epoch time in seconds.
        df['timestamp'] = df['pt'].apply(lambda x: time.gmtime(x/1000))
        df['time_str'] = df['timestamp'].apply(lambda x: time.strftime('%Y-%m-%dT%H:%M:%S', x))

        match = re.search(r'\'eventId\'\:\ \'(\d+)\'', str(df['mc'].iloc[0]))
        eventId = match.group(0)
        eventId = eventId.replace("'", "")
        eventId = eventId.replace("eventId: ", "")
        ltps = []
        inPlays = []
        inPlay = False
        for d in l:

            d['inPlay'] = False

            if 'marketDefinition' in str(d['mc']) and inPlay == False:
                if "'inPlay': True" in str(d['mc']):
                    inPlay = True
                    d['inPlay'] = True
            else:
                if inPlay == False:
                    d['inPlay'] = False
                else:
                    d['inPlay'] = True  
            inPlays.append(d['inPlay'])


            if 'rc' in d['mc'][0].keys():
                prices = d['mc'][0]['rc']
                t = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime(d['pt']/1000))
                for p in prices:
                    if 'hc' not in p.keys():
                        p['hc'] = "None"
                    p['t'] = t
                    p['inPlay'] = d['inPlay']
                ltps.append(prices)


        ltps_sorted = [item for sublist in ltps for item in sublist]  
        df = pd.DataFrame(ltps_sorted)

        df['eventId'] = eventId
        df['marketId'] = file    
        mother_df = mother_df.append(df)

# sort the mcms by time
mother_df = mother_df.sort_values(by=['t'])
mother_df.to_csv('./csvs02/mother_out.csv', index=False)
mother_df.head()

Extract all the market definition messages and save them to a single dataframe (market_defs_df_1), also get the initial market definitions (market_defs_unique)

In [ ]:
# extract all the market definition mcms
# go by market id and count the in play markets
# for those that have in play extract the times when they go in play
# extract the commission rates for each

file_location = './data_unpacked'
dirListing = os.listdir(file_location)
market_defs = []

for subdir in dirListing:
    for file in os.listdir(file_location + '/' + subdir + '/'):
        with open(file_location + '/' + subdir + '/' + file) as f:
            for line in f:
                if("marketDefinition" in line):
                    market_defs.append(json.loads(line))


#put market_defs to a dataframe so that different fields are columns
mcm_df = pd.DataFrame(market_defs[0:])

#now from the mc column
market_defs_df = pd.DataFrame(mcm_df['mc'])

market_defs_df['mc'] = market_defs_df['mc'].astype('str')
market_defs_df_1 = pd.DataFrame(market_defs_df['mc'])
market_defs_df_1['id'] = market_defs_df['mc'].str.extract(r"\'id\'\:\ \'(\d\.\d+)\'", expand=True, flags= re.MULTILINE | re.IGNORECASE)
market_defs_df_1['turnInPlayEnabled'] = market_defs_df['mc'].str.extract(r'\'turnInPlayEnabled\'\:\ ([a-z]+)', expand=True, flags= re.MULTILINE | re.IGNORECASE)
market_defs_df_1['marketBaseRate'] = market_defs_df['mc'].str.extract(r'\'marketBaseRate\'\:\ (\d+\.\d+)', expand=True, flags= re.MULTILINE | re.IGNORECASE)
market_defs_df_1['eventId'] = market_defs_df['mc'].str.extract(r'\'eventId\'\:\ \'(\d+)\'', expand=True, flags= re.MULTILINE | re.IGNORECASE)
market_defs_df_1['marketTime'] = market_defs_df['mc'].str.extract(r'\'marketTime\'\:\ \'(\d+\-\d+\-\d+.\d+\:\d+\:\d+\.\d+.)\'', expand=True, flags= re.MULTILINE | re.IGNORECASE)
market_defs_df_1['suspendTime'] = market_defs_df['mc'].str.extract(r'\'suspendTime\'\:\ \'(\d+\-\d+\-\d+.\d+\:\d+\:\d+\.\d+.)\'', expand=True, flags= re.MULTILINE | re.IGNORECASE)
market_defs_df_1['complete'] = market_defs_df['mc'].str.extract(r'\'complete\'\:\ (\w+)', expand=True, flags= re.MULTILINE | re.IGNORECASE)
market_defs_df_1['inPlay'] = market_defs_df['mc'].str.extract(r'\'inPlay\'\:\ (\w+)', expand=True, flags= re.MULTILINE | re.IGNORECASE)
market_defs_df_1['numberOfActiveRunners'] = market_defs_df['mc'].str.extract(r'\'numberOfActiveRunners\'\:\ (\d+)', expand=True, flags= re.MULTILINE | re.IGNORECASE)
market_defs_df_1['betDelay'] = market_defs_df['mc'].str.extract(r'\'betDelay\'\:\ (\d+)', expand=True, flags= re.MULTILINE | re.IGNORECASE)
market_defs_df_1 = market_defs_df_1.drop(columns='mc')

print(market_defs_df_1.head())
market_defs_df_1.to_csv('./csvs02/market_defs_all.csv', index=False)

market_defs_unique = market_defs_df_1.drop_duplicates('id')
market_defs_unique = market_defs_unique.drop(columns=['inPlay', 'betDelay'])
market_defs_unique.head()
market_defs_unique.to_csv('./csvs02/market_defs_initial.csv', index=False)

Separate the mother_df by events

In [ ]:
#create unique list of names
events = mother_df.eventId.unique()

#create a data frame dictionary to store the data frames
DataFrameDict = {elem : pd.DataFrame() for elem in events}

for key in DataFrameDict.keys():
    DataFrameDict[key] = mother_df[:][mother_df.eventId == key]

for event in events:
    DataFrameDict[event].to_csv("./csvs02/events/" + event + '.csv', index = False)

Extract runners per market and their status as either winners or losers. The last message in the file contains a market definition with the status of all runners. Get the number of active runners from the first message.

In [ ]:
file_location = './data_unpacked'
dirListing = os.listdir(file_location)
market_defs = []

for subdir in dirListing:
    for file in os.listdir(file_location + '/' + subdir + '/'):
        with open(file_location + '/' + subdir + '/' + file) as f:
            first_line = f.readline()
            last_line = f.readlines()[-1]
            market_defs.append(json.loads(first_line))
            market_defs.append(json.loads(last_line))


runners_df = pd.DataFrame(market_defs[0:])
runners_df = pd.DataFrame(runners_df.mc)

runners_df['mc'] = runners_df['mc'].astype('str')
runners_df_1 = pd.DataFrame(runners_df['mc'])
runners_df_1['id'] = runners_df['mc'].str.extract(r"\'id\'\:\ \'(\d\.\d+)\'", expand=True, flags= re.MULTILINE | re.IGNORECASE)
runners_df_1['eventId'] = runners_df['mc'].str.extract(r'\'eventId\'\:\ \'(\d+)\'', expand=True, flags= re.MULTILINE | re.IGNORECASE)
runners_df_1['numberOfActiveRunners'] = runners_df['mc'].str.extract(r'\'numberOfActiveRunners\'\:\ (\d+)', expand=True, flags= re.MULTILINE | re.IGNORECASE)
runners_df_1['winner'] = runners_df['mc'].str.extract(r'\'WINNER\'\,\ \'sortPriority\'\:\ \d+\,\ \'bsp\'\:\ \d+.\d+\,\ \'id\'\:\ (\d+)', expand=True, flags= re.MULTILINE | re.IGNORECASE)
runners_df_1 = runners_df_1.drop(columns='mc')

runners_df_2 = runners_df_1.copy()
runners_df_2.winner = runners_df_2.winner.shift(periods=-1)
runners_df_3 = runners_df_2[runners_df_2['numberOfActiveRunners'] != '0'].copy()
runners_df_3.to_csv('./csvs02/winners.csv', index=False)